In [ ]:
# Locality Lens - OSM Data Exploration
# Fetching all relevant data from OpenStreetMap for Delhi location

# ============================================================================
# FIX SSL ISSUES: Corporate Proxy/Firewall SSL Inspection
# ============================================================================
import os
import ssl
import urllib3

# Disable SSL verification at Python level
ssl._create_default_https_context = ssl._create_unverified_context

# Set environment variables
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['REQUESTS_CA_BUNDLE'] = ''

# Import and patch requests/urllib3 BEFORE OSMnx
import requests
import urllib3.poolmanager

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Patch urllib3 PoolManager (OSMnx uses this)
_original_poolmanager_init = urllib3.poolmanager.PoolManager.__init__

def _patched_poolmanager_init(self, *args, **kwargs):
    """Force SSL verification off for urllib3."""
    kwargs['cert_reqs'] = ssl.CERT_NONE
    kwargs['ca_certs'] = None
    kwargs['ca_cert_dir'] = None
    return _original_poolmanager_init(self, *args, **kwargs)

urllib3.poolmanager.PoolManager.__init__ = _patched_poolmanager_init

# Patch requests at all levels
_original_get = requests.get
_original_post = requests.post
_original_session_request = requests.Session.request

def _patched_get(*args, **kwargs):
    kwargs.setdefault('verify', False)
    return _original_get(*args, **kwargs)

def _patched_post(*args, **kwargs):
    kwargs.setdefault('verify', False)
    return _original_post(*args, **kwargs)

def _patched_session_request(self, *args, **kwargs):
    kwargs.setdefault('verify', False)
    return _original_session_request(self, *args, **kwargs)

requests.get = _patched_get
requests.post = _patched_post
requests.Session.request = _patched_session_request

# Patch HTTPAdapter (what OSMnx actually uses internally)
_original_adapter_send = requests.adapters.HTTPAdapter.send

def _patched_adapter_send(self, request, *args, **kwargs):
    """Patch HTTPAdapter.send - this is what OSMnx uses."""
    kwargs.setdefault('verify', False)
    return _original_adapter_send(self, request, *args, **kwargs)

requests.adapters.HTTPAdapter.send = _patched_adapter_send

# ============================================================================
# NOW import OSMnx and other libraries (after SSL patches)
# ============================================================================
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import folium
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

# Configure OSMnx
ox.settings.log_console = True
ox.settings.use_cache = True
ox.settings.timeout = 300  # Increase timeout for large queries

print("✅ OSMnx configured successfully!")
print("✅ SSL verification disabled for corporate proxy environment")

In [ ]:
# Delhi Coordinate - Using Connaught Place (Central Delhi) as example
# You can change this to any location in Delhi

LATITUDE = 28.6304  # Connaught Place, New Delhi
LONGITUDE = 77.2177
RADIUS = 2000  # 2km in meters

location_point = (LATITUDE, LONGITUDE)

print(f"📍 Location: Connaught Place, New Delhi")
print(f"   Coordinates: {LATITUDE}, {LONGITUDE}")
print(f"   Search Radius: {RADIUS}m (2km)")
print(f"\n🔍 Fetching OSM data...")

In [ ]:
# 1. SCHOOLS - Essential for families
print("📚 Fetching Schools...")
try:
    schools = ox.features_from_point(
        location_point,
        tags={'amenity': 'school'},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(schools)} schools")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    schools = gpd.GeoDataFrame()

In [ ]:
# 2. HOSPITALS & CLINICS - Essential for all
print("🏥 Fetching Hospitals & Clinics...")
try:
    hospitals = ox.features_from_point(
        location_point,
        tags={'amenity': ['hospital', 'clinic', 'doctors', 'dentist']},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(hospitals)} hospitals/clinics")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    hospitals = gpd.GeoDataFrame()

In [ ]:
# 3. PARKS & GARDENS - Green spaces
print("🌳 Fetching Parks & Gardens...")
try:
    parks = ox.features_from_point(
        location_point,
        tags={'leisure': ['park', 'garden', 'recreation_ground']},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(parks)} parks/gardens")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    parks = gpd.GeoDataFrame() 

In [ ]:
# 4. RESTAURANTS & CAFES - Food & dining
print("🍽️ Fetching Restaurants & Cafes...")
try:
    restaurants = ox.features_from_point(
        location_point,
        tags={'amenity': ['restaurant', 'cafe', 'fast_food', 'food_court']},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(restaurants)} restaurants/cafes")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    restaurants = gpd.GeoDataFrame()

In [ ]:
# 5. SHOPS & MARKETS - Shopping
print("🛍️ Fetching Shops & Markets...")
try:
    shops = ox.features_from_point(
        location_point,
        tags={'shop': True},  # All shop types
        dist=RADIUS
    )
    print(f"   ✅ Found {len(shops)} shops")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    shops = gpd.GeoDataFrame()

In [ ]:
# 6. METRO STATIONS - Public transport
print("🚇 Fetching Metro Stations...")
try:
    metro_stations = ox.features_from_point(
        location_point,
        tags={'railway': 'station', 'station': ['subway', 'metro']},
        dist=RADIUS * 2  # Search wider for metro (might be further)
    )
    print(f"   ✅ Found {len(metro_stations)} metro stations")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    metro_stations = gpd.GeoDataFrame()

In [ ]:
# 7. BUS STOPS - Public transport
print("🚌 Fetching Bus Stops...")
try:
    bus_stops = ox.features_from_point(
        location_point,
        tags={'highway': 'bus_stop', 'public_transport': 'platform'},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(bus_stops)} bus stops")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    bus_stops = gpd.GeoDataFrame()

In [ ]:
# 8. BANKS & ATMs - Financial services
print("🏦 Fetching Banks & ATMs...")
try:
    banks = ox.features_from_point(
        location_point,
        tags={'amenity': ['bank', 'atm']},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(banks)} banks/ATMs")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    banks = gpd.GeoDataFrame()

In [ ]:
# 9. GYMS & FITNESS - Lifestyle
print("💪 Fetching Gyms & Fitness Centers...")
try:
    gyms = ox.features_from_point(
        location_point,
        tags={'amenity': ['gym', 'fitness_center'], 'leisure': 'fitness_centre'},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(gyms)} gyms/fitness centers")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    gyms = gpd.GeoDataFrame()

In [ ]:
# 10. BARS & NIGHTLIFE - Entertainment
print("🍺 Fetching Bars & Nightlife...")
try:
    bars = ox.features_from_point(
        location_point,
        tags={'amenity': ['bar', 'pub', 'nightclub']},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(bars)} bars/nightlife venues")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    bars = gpd.GeoDataFrame()

In [ ]:
# 11. TEMPLES & PLACES OF WORSHIP - Cultural
print("🕉️ Fetching Places of Worship...")
try:
    worship = ox.features_from_point(
        location_point,
        tags={'amenity': ['place_of_worship'], 'building': ['temple', 'mosque', 'church', 'gurudwara']},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(worship)} places of worship")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    worship = gpd.GeoDataFrame()

In [ ]:
# 12. ROAD NETWORK - Connectivity analysis
print("🛣️ Fetching Road Network...")
try:
    # Get road network graph
    G = ox.graph_from_point(
        location_point,
        dist=RADIUS,
        network_type='all'  # All road types
    )
    
    # Convert to GeoDataFrame
    nodes, edges = ox.graph_to_gdfs(G)
    print(f"   ✅ Found {len(edges)} road segments")
    print(f"   ✅ Found {len(nodes)} road nodes")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    edges = gpd.GeoDataFrame()
    nodes = gpd.GeoDataFrame()

In [ ]:
# 13. LIBRARIES - Educational
print("📖 Fetching Libraries...")
try:
    libraries = ox.features_from_point(
        location_point,
        tags={'amenity': 'library'},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(libraries)} libraries")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    libraries = gpd.GeoDataFrame()

In [ ]:
# 14. PHARMACIES - Healthcare
print("💊 Fetching Pharmacies...")
try:
    pharmacies = ox.features_from_point(
        location_point,
        tags={'amenity': 'pharmacy', 'shop': 'pharmacy'},
        dist=RADIUS
    )
    print(f"   ✅ Found {len(pharmacies)} pharmacies")
except Exception as e:
    print(f"   ⚠️ Error: {e}")
    pharmacies = gpd.GeoDataFrame()

In [ ]:
# SUMMARY OF ALL DATA FETCHED
print("\n" + "="*60)
print("📊 DATA FETCHING SUMMARY")
print("="*60)

data_summary = {
    'Category': [
        'Schools', 'Hospitals/Clinics', 'Parks/Gardens', 
        'Restaurants/Cafes', 'Shops', 'Metro Stations',
        'Bus Stops', 'Banks/ATMs', 'Gyms', 'Bars/Nightlife',
        'Places of Worship', 'Libraries', 'Pharmacies', 'Road Segments'
    ],
    'Count': [
        len(schools), len(hospitals), len(parks),
        len(restaurants), len(shops), len(metro_stations),
        len(bus_stops), len(banks), len(gyms), len(bars),
        len(worship), len(libraries), len(pharmacies), len(edges)
    ]
}

summary_df = pd.DataFrame(data_summary)
print(summary_df.to_string(index=False))
print("\n✅ All data fetched successfully!")

In [ ]:
# EXPLORE DATA STRUCTURE - Example with Schools
if len(schools) > 0:
    print("\n📚 Example: Schools Data Structure")
    print("="*60)
    print(f"Columns: {list(schools.columns)}")
    print(f"\nFirst few schools:")
    # Show key columns if they exist
    key_cols = ['name', 'amenity', 'geometry']
    available_cols = [col for col in key_cols if col in schools.columns]
    if available_cols:
        print(schools[available_cols].head())
    else:
        print(schools.head())

In [ ]:
# VISUALIZE ON MAP
print("\n🗺️ Creating interactive map...")

# Create base map
m = folium.Map(
    location=[LATITUDE, LONGITUDE],
    zoom_start=14,
    tiles='OpenStreetMap'
)

# Add center point
folium.Marker(
    [LATITUDE, LONGITUDE],
    popup='Center Point',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Add schools
if len(schools) > 0 and 'geometry' in schools.columns:
    for idx, row in schools.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"School: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='blue', icon='book', prefix='fa')
            ).add_to(m)

# Add hospitals
if len(hospitals) > 0 and 'geometry' in hospitals.columns:
    for idx, row in hospitals.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Hospital: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='red', icon='plus', prefix='fa')
            ).add_to(m)

# Add parks (as polygons/areas)
if len(parks) > 0 and 'geometry' in parks.columns:
    for idx, row in parks.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Park: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='green', icon='tree', prefix='fa')
            ).add_to(m)

# Add restaurants
if len(restaurants) > 0 and 'geometry' in restaurants.columns:
    for idx, row in restaurants.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Restaurant: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='orange', icon='cutlery', prefix='fa')
            ).add_to(m)

# Add metro stations
if len(metro_stations) > 0 and 'geometry' in metro_stations.columns:
    for idx, row in metro_stations.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Metro: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='purple', icon='train', prefix='fa')
            ).add_to(m)

# Add search radius circle
folium.Circle(
    location=[LATITUDE, LONGITUDE],
    radius=RADIUS,
    popup=f'{RADIUS}m radius',
    color='blue',
    fill=False,
    weight=2
).add_to(m)

# Display map
print("✅ Map created! Displaying...")

# Save map to HTML and display
from IPython.display import HTML, display
import tempfile
import os

# Save to temporary file
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.html', mode='w')
m.save(temp_file.name)
temp_file.close()

# VISUALIZE ON MAP
print("\n🗺️ Creating interactive map...")

# Create base map
m = folium.Map(
    location=[LATITUDE, LONGITUDE],
    zoom_start=14,
    tiles='OpenStreetMap'
)

# Add center point
folium.Marker(
    [LATITUDE, LONGITUDE],
    popup='Center Point',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Add schools
if len(schools) > 0 and 'geometry' in schools.columns:
    for idx, row in schools.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"School: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='blue', icon='book', prefix='fa')
            ).add_to(m)

# Add hospitals
if len(hospitals) > 0 and 'geometry' in hospitals.columns:
    for idx, row in hospitals.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Hospital: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='red', icon='plus', prefix='fa')
            ).add_to(m)

# Add parks (as polygons/areas)
if len(parks) > 0 and 'geometry' in parks.columns:
    for idx, row in parks.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Park: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='green', icon='tree', prefix='fa')
            ).add_to(m)

# Add restaurants
if len(restaurants) > 0 and 'geometry' in restaurants.columns:
    for idx, row in restaurants.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Restaurant: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='orange', icon='cutlery', prefix='fa')
            ).add_to(m)

# Add metro stations
if len(metro_stations) > 0 and 'geometry' in metro_stations.columns:
    for idx, row in metro_stations.iterrows():
        if hasattr(row.geometry, 'centroid'):
            folium.Marker(
                [row.geometry.centroid.y, row.geometry.centroid.x],
                popup=f"Metro: {row.get('name', 'Unknown')}",
                icon=folium.Icon(color='purple', icon='train', prefix='fa')
            ).add_to(m)

# Add search radius circle
folium.Circle(
    location=[LATITUDE, LONGITUDE],
    radius=RADIUS,
    popup=f'{RADIUS}m radius',
    color='blue',
    fill=False,
    weight=2
).add_to(m)

# Display map using HTML representation (works without trust)
print("✅ Map created! Displaying...")
from IPython.display import HTML, display

# Get HTML representation
map_html = m._repr_html_()

# Display it
display(HTML(map_html))

In [ ]:
# CALCULATE BASIC STATISTICS
print("\n📊 CALCULATING STATISTICS")
print("="*60)

from shapely.geometry import Point
from geopy.distance import geodesic

center_point = Point(LONGITUDE, LATITUDE)

# Calculate distances for metro stations
if len(metro_stations) > 0 and 'geometry' in metro_stations.columns:
    metro_distances = []
    for idx, row in metro_stations.iterrows():
        if hasattr(row.geometry, 'centroid'):
            metro_geom = row.geometry.centroid
            distance = geodesic(
                (LATITUDE, LONGITUDE),
                (metro_geom.y, metro_geom.x)
            ).meters
            metro_distances.append(distance)
    
    if metro_distances:
        nearest_metro = min(metro_distances) / 1000  # Convert to km
        print(f"🚇 Nearest Metro Station: {nearest_metro:.2f} km")
    else:
        print("🚇 Nearest Metro Station: Not found in 2km radius")

# Calculate park area
if len(parks) > 0 and 'geometry' in parks.columns:
    total_park_area = 0
    for idx, row in parks.iterrows():
        if hasattr(row.geometry, 'area'):
            # Convert from square meters to square km
            area_km2 = row.geometry.area / 1_000_000
            total_park_area += area_km2
    print(f"🌳 Total Park Area: {total_park_area:.2f} km²")
else:
    print("🌳 Total Park Area: 0 km²")

# Calculate road density
if len(edges) > 0 and 'geometry' in edges.columns:
    total_road_length = 0
    for idx, row in edges.iterrows():
        if hasattr(row.geometry, 'length'):
            total_road_length += row.geometry.length  # in meters
    
    # Area of circle with radius RADIUS
    area_km2 = (np.pi * (RADIUS ** 2)) / 1_000_000
    road_density = total_road_length / 1000 / area_km2  # km per km²
    print(f"🛣️ Road Density: {road_density:.2f} km/km²")
else:
    print("🛣️ Road Density: 0 km/km²")

# POI Density
total_pois = (
    len(schools) + len(hospitals) + len(parks) + len(restaurants) + 
    len(shops) + len(banks) + len(gyms) + len(bars) + len(worship)
)
poi_density = total_pois / area_km2
print(f"📍 POI Density: {poi_density:.2f} POIs/km²")

print("\n✅ Statistics calculated!")

In [ ]:
# ROBUST AFFORDABILITY ANALYSIS FOR RENTERS/RESIDENTS
print("\n💰 Analyzing Affordability for Renters/Residents...")
print("="*60)

def calculate_affordability_robust(location_point, radius, restaurants, shops, bars, banks, 
                                   metro_stations, parks, edges):
    """
    ROBUST AFFORDABILITY LOGIC FOR RENTERS
    
    Logic:
    1. Commercial Activity (40% weight) - More commercial = more expensive
       - High commercial density = premium location
       - Bank density = commercial hub = expensive
       - Office buildings = business district = expensive
    
    2. POI Type Analysis (30% weight) - What people spend on
       - Fine dining ratio = expensive lifestyle
       - Luxury shops = high purchasing power area
       - Fast food ratio = budget-friendly area
    
    3. Connectivity Premium (20% weight) - Location value
       - Metro proximity = premium pricing
       - Good connectivity = desirable = expensive
    
    4. Housing Indicators (10% weight) - Residential character
       - High residential = might be more affordable
       - But premium residential = still expensive
    
    Score: 0-100 where:
    - 0-25: Very Expensive (like CP, premium areas)
    - 25-45: Expensive
    - 45-60: Moderate
    - 60-75: Affordable
    - 75-100: Very Affordable
    """
    
    area_km2 = (3.14159 * (radius ** 2)) / 1_000_000
    
    # ============================================
    # 1. COMMERCIAL ACTIVITY SCORE (40% weight)
    # ============================================
    commercial_score = 0  # Higher = more commercial = more expensive
    
    # Bank density (strong indicator of commercial area)
    if len(banks) > 0:
        bank_density = len(banks) / area_km2
        if bank_density > 20:  # Very high (like CP)
            commercial_score += 40
        elif bank_density > 10:
            commercial_score += 25
        elif bank_density > 5:
            commercial_score += 15
        else:
            commercial_score += 5
    
    # Restaurant density (commercial activity)
    if len(restaurants) > 0:
        restaurant_density = len(restaurants) / area_km2
        if restaurant_density > 50:  # Very high commercial
            commercial_score += 20
        elif restaurant_density > 30:
            commercial_score += 12
        elif restaurant_density > 15:
            commercial_score += 6
    
    # Shop density (commercial activity)
    if len(shops) > 0:
        shop_density = len(shops) / area_km2
        if shop_density > 100:  # Very high
            commercial_score += 15
        elif shop_density > 50:
            commercial_score += 8
    
    # Nightlife (bars/clubs = commercial entertainment district)
    if len(bars) > 0:
        bar_density = len(bars) / area_km2
        if bar_density > 5:
            commercial_score += 10
        elif bar_density > 2:
            commercial_score += 5
    
    commercial_score = min(100, commercial_score)  # Cap at 100
    
    # ============================================
    # 2. POI TYPE ANALYSIS (30% weight)
    # ============================================
    luxury_score = 0
    budget_score = 0
    
    # Restaurant type analysis
    if len(restaurants) > 0:
        fine_dining_count = 0
        fast_food_count = 0
        total_restaurants = len(restaurants)
        
        for idx, row in restaurants.iterrows():
            name = str(row.get('name', '')).lower()
            amenity = str(row.get('amenity', '')).lower()
            cuisine = str(row.get('cuisine', '')).lower()
            
            # Fine dining indicators
            fine_dining_keywords = [
                'restaurant', 'fine', 'dining', 'bistro', 'cafe', 'lounge',
                'steakhouse', 'brasserie', 'gourmet', 'cuisine', 'grill',
                'italian', 'french', 'japanese', 'sushi', 'continental'
            ]
            if any(kw in name for kw in fine_dining_keywords) or any(c in cuisine for c in ['french', 'italian', 'japanese', 'continental']):
                fine_dining_count += 1
            
            # Budget indicators
            if amenity == 'fast_food' or 'fast' in name or 'street' in name or 'food_court' in name:
                fast_food_count += 1
        
        # Calculate ratios
        fine_dining_ratio = fine_dining_count / total_restaurants if total_restaurants > 0 else 0
        fast_food_ratio = fast_food_count / total_restaurants if total_restaurants > 0 else 0
        
        luxury_score += fine_dining_ratio * 50  # Up to 50 points
        budget_score += fast_food_ratio * 30     # Up to 30 points
    
    # Shop type analysis
    if len(shops) > 0:
        luxury_shops = 0
        budget_shops = 0
        total_shops = len(shops)
        
        for idx, row in shops.iterrows():
            shop_type = str(row.get('shop', '')).lower()
            name = str(row.get('name', '')).lower()
            
            # Luxury shop indicators
            luxury_keywords = [
                'jewelry', 'watches', 'luxury', 'boutique', 'fashion',
                'designer', 'premium', 'exclusive', 'brand', 'outlet'
            ]
            if any(kw in shop_type or kw in name for kw in luxury_keywords):
                luxury_shops += 1
            
            # Budget shop indicators
            budget_keywords = [
                'convenience', 'supermarket', 'discount', 'wholesale',
                'mart', 'bazaar', 'market', 'store', 'general'
            ]
            if any(kw in shop_type or kw in name for kw in budget_keywords):
                budget_shops += 1
        
        luxury_ratio = luxury_shops / total_shops if total_shops > 0 else 0
        budget_ratio = budget_shops / total_shops if total_shops > 0 else 0
        
        luxury_score += luxury_ratio * 30  # Up to 30 points
        budget_score += budget_ratio * 20  # Up to 20 points
    
    # Normalize POI scores
    poi_score = luxury_score - budget_score  # Positive = expensive, Negative = affordable
    poi_score = max(-50, min(50, poi_score))  # Range: -50 to +50
    
    # ============================================
    # 3. CONNECTIVITY PREMIUM (20% weight)
    # ============================================
    connectivity_score = 0
    
    # Metro stations (premium pricing near metro)
    if len(metro_stations) > 0:
        # Very close metro = premium
        if len(metro_stations) >= 2:  # Multiple metro stations nearby
            connectivity_score += 20
        elif len(metro_stations) == 1:
            connectivity_score += 10
    
    # Bus stops (moderate premium)
    if len(bus_stops) > 0:
        bus_density = len(bus_stops) / area_km2
        if bus_density > 30:
            connectivity_score += 5
    
    # Road density (good connectivity = premium)
    if len(edges) > 0:
        total_road_length = sum(edge.geometry.length for edge in edges.itertuples() 
                              if hasattr(edge, 'geometry') and hasattr(edge.geometry, 'length'))
        road_density = (total_road_length / 1000) / area_km2  # km/km²
        if road_density > 12:  # Very well connected
            connectivity_score += 5
    
    connectivity_score = min(20, connectivity_score)  # Cap at 20
    
    # ============================================
    # 4. HOUSING INDICATORS (10% weight)
    # ============================================
    # Try to fetch residential data (lightweight check)
    housing_score = 0
    try:
        # Quick check for residential areas
        residential = ox.features_from_point(
            location_point,
            tags={'landuse': 'residential'},
            dist=radius
        )
        if len(residential) > 0:
            # High residential = might be more affordable (but not always)
            # We'll use this as a slight negative to commercial score
            residential_ratio = len(residential) / (len(residential) + commercial_score/10)
            if residential_ratio > 0.7:  # Mostly residential
                housing_score = -5  # Slight discount (more affordable)
    except:
        pass  # Skip if fails
    
    # ============================================
    # FINAL CALCULATION
    # ============================================
    
    # Weighted combination
    # Commercial: 40%, POI: 30%, Connectivity: 20%, Housing: 10%
    weighted_score = (
        (commercial_score * 0.4) +      # Commercial activity
        ((50 + poi_score) * 0.3) +      # POI types (normalize to 0-100)
        (connectivity_score * 5) +      # Connectivity (scale up)
        (50 + housing_score)             # Housing adjustment
    )
    
    # Normalize to 0-100 scale
    # Higher commercial + luxury + connectivity = lower affordability (more expensive)
    # We want: high score = affordable, low score = expensive
    affordability_score = 100 - weighted_score
    affordability_score = max(0, min(100, affordability_score))
    
    # Calculate category
    if affordability_score <= 25:
        category = "Very Expensive 💰💰💰💰💰"
        rent_range = "₹30,000+ /month (1BHK)"
    elif affordability_score <= 45:
        category = "Expensive 💰💰💰💰"
        rent_range = "₹20,000-30,000 /month (1BHK)"
    elif affordability_score <= 60:
        category = "Moderate 💰💰💰"
        rent_range = "₹12,000-20,000 /month (1BHK)"
    elif affordability_score <= 75:
        category = "Affordable 💰💰"
        rent_range = "₹8,000-12,000 /month (1BHK)"
    else:
        category = "Very Affordable 💰"
        rent_range = "₹5,000-8,000 /month (1BHK)"
    
    return {
        'score': round(affordability_score, 1),
        'category': category,
        'rent_range': rent_range,
        'breakdown': {
            'commercial_activity': round(commercial_score, 1),
            'poi_luxury_score': round(luxury_score, 1),
            'poi_budget_score': round(budget_score, 1),
            'connectivity_premium': round(connectivity_score, 1),
            'housing_adjustment': round(housing_score, 1)
        },
        'indicators': {
            'bank_density': round(len(banks) / area_km2, 1) if len(banks) > 0 else 0,
            'restaurant_density': round(len(restaurants) / area_km2, 1) if len(restaurants) > 0 else 0,
            'metro_stations': len(metro_stations),
            'fine_dining_ratio': round(fine_dining_count / len(restaurants), 2) if len(restaurants) > 0 else 0,
            'fast_food_ratio': round(fast_food_count / len(restaurants), 2) if len(restaurants) > 0 else 0
        }
    }

# Calculate affordability
affordability = calculate_affordability_robust(
    location_point, RADIUS, restaurants, shops, bars, banks,
    metro_stations, parks, edges
)

print(f"\n💰 AFFORDABILITY ANALYSIS RESULTS")
print("="*60)
print(f"   Score: {affordability['score']}/100")
print(f"   Category: {affordability['category']}")
print(f"   Estimated Rent Range: {affordability['rent_range']}")
print(f"\n   📊 Breakdown:")
print(f"      • Commercial Activity: {affordability['breakdown']['commercial_activity']}/100")
print(f"      • Luxury POI Score: {affordability['breakdown']['poi_luxury_score']}")
print(f"      • Budget POI Score: {affordability['breakdown']['poi_budget_score']}")
print(f"      • Connectivity Premium: {affordability['breakdown']['connectivity_premium']}/20")
print(f"      • Housing Adjustment: {affordability['breakdown']['housing_adjustment']}")
print(f"\n   📈 Key Indicators:")
print(f"      • Bank Density: {affordability['indicators']['bank_density']} per km²")
print(f"      • Restaurant Density: {affordability['indicators']['restaurant_density']} per km²")
print(f"      • Metro Stations: {affordability['indicators']['metro_stations']}")
print(f"      • Fine Dining Ratio: {affordability['indicators']['fine_dining_ratio']*100:.1f}%")
print(f"      • Fast Food Ratio: {affordability['indicators']['fast_food_ratio']*100:.1f}%")

In [ ]:
# AQI ANALYSIS - Complete Fix with Detailed Error Handling
print("\n🌬️ Fetching Air Quality Data...")
print("="*60)

import os
from pathlib import Path
from dotenv import load_dotenv
import requests
import json

# Load .env file - try multiple paths
env_loaded = False
env_paths = [
    Path('../.env'),
    Path('.env'),
    Path('../locality-lens/.env')
]

for env_path in env_paths:
    if env_path.exists():
        load_dotenv(dotenv_path=env_path)
        print(f"   ✅ Loaded .env from: {env_path}")
        env_loaded = True
        break

if not env_loaded:
    # Try default location
    load_dotenv()
    print("   ⚠️ Using default .env location")

def get_aqi_openweather_fixed(lat, lon, api_key):
    """
    Fixed AQI function with comprehensive error handling
    """
    if not api_key:
        print("   ❌ API key is None or empty")
        return None
    
    # Check key format (should be 32 chars)
    if len(api_key) < 20:
        print(f"   ⚠️ API key seems too short ({len(api_key)} chars)")
        print("   💡 OpenWeatherMap API keys are usually 32 characters")
    
    url = "https://api.openweathermap.org/data/2.5/air_pollution"
    params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key
    }
    
    print(f"   🔍 Calling: {url}")
    print(f"   📍 Location: {lat}, {lon}")
    print(f"   🔑 API Key: {api_key[:8]}...{api_key[-4:] if len(api_key) > 12 else '***'}")
    
    try:
        response = requests.get(url, params=params, timeout=15)
        
        print(f"   📡 Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            
            if 'list' not in data or len(data['list']) == 0:
                print("   ⚠️ No AQI data in response")
                return None
            
            aqi_data = data['list'][0]
            main = aqi_data.get('main', {})
            components = aqi_data.get('components', {})
            
            aqi_value = main.get('aqi', 0)
            aqi_mapping = {1: 50, 2: 100, 3: 150, 4: 200, 5: 300}
            standard_aqi = aqi_mapping.get(aqi_value, aqi_value * 50)
            
            result = {
                'aqi': standard_aqi,
                'aqi_level': aqi_value,
                'category': get_aqi_category(standard_aqi),
                'pm25': round(components.get('pm2_5', 0), 1),
                'pm10': round(components.get('pm10', 0), 1),
                'no2': round(components.get('no2', 0), 1),
                'o3': round(components.get('o3', 0), 1)
            }
            
            print("   ✅ AQI data retrieved successfully!")
            return result
            
        elif response.status_code == 401:
            print("\n   ❌ ERROR 401: Unauthorized")
            print("   " + "="*50)
            print("   Possible reasons:")
            print("   1. ❌ API key not activated yet")
            print("      → Wait 2-3 hours after signup")
            print("   2. ❌ Wrong API key")
            print("      → Check your key at: https://home.openweathermap.org/api_keys")
            print("   3. ❌ Air Pollution API not subscribed")
            print("      → This is a SEPARATE subscription (even on free tier)")
            print("      → Go to: https://openweathermap.org/api/air-pollution")
            print("      → Click 'Subscribe' (it's FREE)")
            print("   4. ❌ API key not included in request")
            print("      → Check .env file format: OPENWEATHER_API_KEY=your_key")
            print("   " + "="*50)
            
            # Try to get error details
            try:
                error_data = response.json()
                print(f"   📄 API Response: {json.dumps(error_data, indent=2)}")
            except:
                print(f"   📄 Response: {response.text[:500]}")
            
            print("\n   💡 SOLUTION:")
            print("   1. Go to: https://openweathermap.org/api/air-pollution")
            print("   2. Click 'Get API Key and Start' or 'Subscribe'")
            print("   3. Select 'Free' plan")
            print("   4. Use the SAME API key (it works for all their APIs)")
            print("   5. Wait 2-3 hours for activation")
            print("   6. Then try again")
            
            return None
            
        elif response.status_code == 429:
            print("   ⚠️ Rate limit exceeded - try again later")
            return None
        else:
            print(f"   ❌ Error {response.status_code}")
            try:
                error_data = response.json()
                print(f"   📄 Error: {json.dumps(error_data, indent=2)}")
            except:
                print(f"   📄 Response: {response.text[:500]}")
            return None
            
    except requests.exceptions.Timeout:
        print("   ⚠️ Request timeout")
        return None
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

def get_aqi_category(aqi):
    """Categorize AQI"""
    if aqi <= 50:
        return "Good 🟢"
    elif aqi <= 100:
        return "Moderate 🟡"
    elif aqi <= 150:
        return "Unhealthy for Sensitive Groups 🟠"
    elif aqi <= 200:
        return "Unhealthy 🔴"
    elif aqi <= 300:
        return "Very Unhealthy 🟣"
    else:
        return "Hazardous ⚫"

# Get API key
openweather_key = os.getenv('OPENWEATHER_API_KEY')

if not openweather_key:
    print("   ❌ OPENWEATHER_API_KEY not found")
    print("   💡 Add to .env file: OPENWEATHER_API_KEY=your_key_here")
else:
    # Fetch AQI
    aqi_data = get_aqi_openweather_fixed(LATITUDE, LONGITUDE, openweather_key)
    
    if aqi_data:
        print(f"\n   🌬️ AQI: {aqi_data['aqi']} ({aqi_data['category']})")
        print(f"   📊 PM2.5: {aqi_data['pm25']} μg/m³")
        print(f"   📊 PM10: {aqi_data['pm10']} μg/m³")
        print(f"   📊 NO₂: {aqi_data['no2']} μg/m³")
        print(f"   📊 O₃: {aqi_data['o3']} μg/m³")
    else:
        print("\n   ❌ Could not fetch AQI data")
        print("   💡 See error messages above for troubleshooting")